In [0]:
# v0: XLSX -> CSV (abas Carga1 e Carga2) sem libs externas
import os, io, csv, re, zipfile, xml.etree.ElementTree as ET
from urllib.request import urlopen
from pyspark.sql import functions as F

spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")

In [0]:
# Download do XLSX
xlsx_url = "https://raw.githubusercontent.com/JDLNas/curso_formacao_microsoft_powerbi_profissional/main/Fontes/EXCEL/000_COMPLETAR.xlsx"
processar_path = "/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processar/000_COMPLETAR.xlsx"

with open(processar_path, "wb") as f:
    f.write(requests.get(xlsx_url).content)

# Use Spark to read XLSX and write CSV (no need for openpyxl or pandas)
for sheet in ['Carga1', 'Carga2']:
    df = spark.read.format("com.crealytics.spark.excel") \
        .option("header", "true") \
        .option("dataAddress", f"'{sheet}'!A1") \
        .load(processar_path)
    csv_path = f"/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processados/{sheet}.csv"
    df.write.mode("overwrite").option("header", "true").csv(csv_path)

In [0]:
# ---- Parâmetros ----
xlsx_url = "https://raw.githubusercontent.com/JDLNas/curso_formacao_microsoft_powerbi_profissional/main/Fontes/EXCEL/000_COMPLETAR.xlsx"
processar_path = "/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processar/000_COMPLETAR.xlsx"
processados_dir = "/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processados"
file_stem = os.path.splitext(os.path.basename(processar_path))[0]  # 000_COMPLETAR

# ---- Baixar XLSX ----
local_xlsx = f"{processar_path}"
os.makedirs(os.path.dirname(local_xlsx), exist_ok=True)
with open(local_xlsx, "wb") as f:
    f.write(urlopen(xlsx_url).read())

# ---- Preparar saída ----
out_dir_local = f"{processados_dir}"
os.makedirs(out_dir_local, exist_ok=True)

# ---- Namespaces do Excel ----
NS = {'m': 'http://schemas.openxmlformats.org/spreadsheetml/2006/main'}

# ---- Abrir XLSX (é um .zip) ----
with zipfile.ZipFile(local_xlsx) as z:
    # sharedStrings (texto de células 's')
    shared = []
    if "xl/sharedStrings.xml" in z.namelist():
        ss_xml = ET.fromstring(z.read("xl/sharedStrings.xml"))
        for si in ss_xml.findall("m:si", NS):
            chunks = []
            t = si.find("m:t", NS)
            if t is not None:
                chunks.append(t.text or "")
            else:
                for r in si.findall("m:r", NS):
                    rt = r.find("m:t", NS)
                    if rt is not None:
                        chunks.append(rt.text or "")
            shared.append("".join(chunks))

    # Mapeia nome de aba -> path do worksheet
    wb_xml   = ET.fromstring(z.read("xl/workbook.xml"))
    rels_xml = ET.fromstring(z.read("xl/_rels/workbook.xml.rels"))
    rid_to_target = {rel.attrib['Id']: rel.attrib['Target']
                     for rel in rels_xml.findall('.//{http://schemas.openxmlformats.org/package/2006/relationships}Relationship')}
    sheet_to_path = {}
    for sh in wb_xml.findall("m:sheets/m:sheet", NS):
        name = sh.attrib["name"]
        rid  = sh.attrib['{http://schemas.openxmlformats.org/officeDocument/2006/relationships}id']
        target = rid_to_target[rid]
        sheet_to_path[name] = f"xl/{target}" if not target.startswith("xl/") else target

    def col_letters_to_idx(letters: str) -> int:
        n = 0
        for ch in letters:
            n = n*26 + (ord(ch)-64)  # 'A'->1
        return n-1

    def export_sheet(sheet_name: str, out_csv_path: str):
        if sheet_name not in sheet_to_path:
            raise FileNotFoundError(f"Aba '{sheet_name}' não encontrada no XLSX.")
        ws = ET.fromstring(z.read(sheet_to_path[sheet_name]))

        rows, max_col = [], 0
        for row in ws.findall(".//m:sheetData/m:row", NS):
            vals = {}
            for c in row.findall("m:c", NS):
                ref = c.attrib.get("r","A1")  # p.ex. C5
                m = re.match(r"([A-Z]+)(\d+)", ref)
                col_idx = col_letters_to_idx(m.group(1)) if m else 0
                ctype = c.attrib.get("t")
                v = c.find("m:v", NS)
                is_node = c.find("m:is/m:t", NS)

                val = ""
                if is_node is not None:
                    val = is_node.text or ""
                elif v is not None:
                    raw = v.text or ""
                    if ctype == "s":                 # sharedStrings
                        i = int(raw) if raw.isdigit() else -1
                        val = shared[i] if 0 <= i < len(shared) else ""
                    elif ctype == "b":               # boolean
                        val = "TRUE" if raw == "1" else "FALSE"
                    else:                            # número/string/fórmula (valor em cache)
                        val = raw

                vals[col_idx] = val
                if col_idx+1 > max_col: max_col = col_idx+1
            rows.append([vals.get(i, "") for i in range(max_col)])

        header = rows[0] if rows else []
        body = rows[1:] if len(rows) > 1 else []

        with open(out_csv_path, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            if header: w.writerow(header)
            for r in body:
                if len(r) < max_col: r = r + [""]*(max_col-len(r))
                w.writerow(r)

    # Exporta as duas abas
    export_sheet("Carga1", f"{out_dir_local}/{file_stem}__Carga1.csv")
    export_sheet("Carga2", f"{out_dir_local}/{file_stem}__Carga2.csv")

print("CSVs salvos em:")
print(f" - {processados_dir}/{file_stem}__Carga1.csv")
print(f" - {processados_dir}/{file_stem}__Carga2.csv")

In [0]:
base = f"{processados_dir}"
df_carga1 = (spark.read
                  .option("header","true").csv(f"{base}/{file_stem}__Carga1.csv")
                  .withColumn('_ingest_ts_utc', F.current_timestamp())
                  .withColumn("_ingest_date", F.to_date(F.col("_ingest_ts_utc")))
                  .withColumn("_source_path", F.col("_metadata.file_path"))
                  .withColumn("_source_file", F.regexp_extract(F.col("_source_path"), r"([^/]+)$", 1))
                  .withColumnRenamed("Dados do PIB", "Dados_do_PIB"))


df_carga2 = (spark.read
                  .option("header","true").csv(f"{base}/{file_stem}__Carga2.csv")
                  .withColumn('_ingest_ts_utc', F.current_timestamp())
                  .withColumn("_ingest_date", F.to_date(F.col("_ingest_ts_utc")))
                  .withColumn("_source_path", F.col("_metadata.file_path"))
                  .withColumn("_source_file", F.regexp_extract(F.col("_source_path"), r"([^/]+)$", 1)))


# display(df_carga1); display(df_carga2)

In [0]:
# Define nomes das tabelas Delta para cada carga
tbl_carga1 = '__Carga1'
tbl_carga2 = '__Carga2'

# Define caminhos de destino para as tabelas Delta
destino_path_carga1 = f'fomacao_microsoft_power_bi_profisional.bronze.{tbl_carga1}'
destino_path_carga2 = f'fomacao_microsoft_power_bi_profisional.bronze.{tbl_carga2}'

# Escreve df_carga1 como tabela Delta particionada por _ingest_date, sobrescrevendo se já existir
df_bronze = (df_carga1.write
             .format('delta')
             .mode('overwrite')
             .option('overwriteSchema', 'true')
             .partitionBy('_ingest_date')
             .saveAsTable(destino_path_carga1))

# Escreve df_carga2 como tabela Delta particionada por _ingest_date, sobrescrevendo se já existir
df_bronze = (df_carga2.write
             .format('delta')
             .mode('overwrite')
             .option('overwriteSchema', 'true')
             .partitionBy('_ingest_date')
             .saveAsTable(destino_path_carga2))

# Exibe a contagem de registros em cada tabela Delta criada
print(f'Contagem de registros: {spark.table(destino_path_carga1).count()}')
print(f'Contagem de registros: {spark.table(destino_path_carga2).count()}')